In [1]:
import json
import pipeline_utils
import os

In [ ]:
root_dir = os.getcwd()
save_dir = "/mnt/home/hlethi/ceph/learningGeometry_20240627"
use_step = False # if `True`, then `num_epoch` equals to number of gradient steps. if `False`, then `num_epoch` equals to number of epoch

# Specify parameter inputs
# TODO: Add comments for each input parameters
parameter_inputs = {
    "model_training": {
        "model": ["VGG11"],
        "task": ["CIFAR-10"],
        "optim": ["SGD"],
        "lr": [1.0],
        "criterion": ["mse"], 
        "num_epoch": [101], # measure statistics at epoch 0 and train 100 epochs
        "save_folder": [save_dir],
        "precision": ["double"],
        "alpha": [1.0, 1.5, 2.0, 3.0, 5.0, 7.5, 10.0, 15.0, 30.0, 50.0, 75.0, 100.0],
        "gain": [1.0],
        "decay": [0.0],
        "useStep": [use_step],
        "seed": [7, 13, 17, 37, 47],
        },
    "feature_analysis": {
        "layers": [["view"]],
        "sampleFunc": ["random"],
        "numSmpl": [50],
        "numCls": [10],
        "fsData": ["CIFAR-10"],
        "subSeed": [1, 4, 18, 27, 29, 49, 64, 82, 90, 98],
        "lastEpoch": [False],
        "saveFeature": [True],
        "runAnalysis": [False],
        "useStep": [use_step],
    },
    "linear_probe": {
        "fsData": ["CIFAR-100"],
        "lastEpoch": [True],
        "sampleFunc": ["random"],
        "numSmpl": [50],
        "numCls": [10],
        "layer": ["view"],
        "runCapacity": [True],
        "runLinearProbe": [True],
        "subSeed": [1, 4, 18, 27, 29, 49, 64, 82, 90, 98],
        "useStep": [use_step],
    },
    "eval_corrupt": {
        "fsData": [["gaussian_noise"], ["defocus_blur"], ["frost"], ["brightness"], ["pixelate"], ["shot_noise"], ["zoom_blur"], ["jpeg_compression"], ["impulse_noise"], ["glass_blur"], ["motion_blur"], ["snow"], ["fog"], ["contrast"], ["elastic_transform"]],
        "sampleFunc": ["random"],
        "numSmpl": [50],
        "numCls": [10],
        "layer": ["view"],
        "runCapacity": [True],
        "lastEpoch": [True],
        "subSeed": [1, 4, 18, 27, 29, 49, 64, 82, 90, 98],
        "useStep": [use_step],
    },
}

In [4]:
# Generate parameter list
parameter_list = pipeline_utils.generate_parameter_pipeline(parameter_inputs, "standard")
for step in parameter_list:
    print(f"Step {step}: Generated a list of {len(parameter_list[step])} parameters!")

Step model_training: Generated a list of 60 parameters!
Step feature_analysis: Generated a list of 600 parameters!


In [5]:
# Save parameter list into json file
filename = "params/params_test.json"
json.dump(parameter_list, open(filename, "w"))
print(f"Save parameter list to json file at {filename}")

Save parameter list to json file at params/params_test.json


In [6]:
n_thread = 4
for step in parameter_list:
    num_jobs = len(parameter_list[step])
    job_array_length = (num_jobs // n_thread) - 1 if num_jobs % n_thread == 0 else num_jobs // n_thread
    print(f"Step {step}: Job array length: 0-{job_array_length}")

Step model_training: Job array length: 0-14
Step feature_analysis: Job array length: 0-149
